# 1.3 신경망의 학습
신경망으로 좋은 추론을 하기 위해서는 학습을 먼저 수행하고, 학습된 매개변수를 이용해 추론을 수행하는 흐름이 일반적이다.

한편 신경망의 학습은 최적의 매개변수값을 찾는 작업이다.

이번절에서는 신경망의 학습에 대해 살펴본다.

## 1.3.1 손실 함수

신경망 학습에는 학습이 얼마나 잘 이루어지고 있는지를 나타내기 위한 '척도'가 필요하다.<br>
일반적으로 학습 단계의 특정 시점에서 신경망의 성능을 나타내는 척도로 **손실(Loss)**를 사용한다.<br>
학습시 주어진 정답 데이터와 신경망이 예측한 결과를 비교해 예측이 얼마나 나쁜가를 산출한 단일 스칼라값(loss)를 구할 수 있다.


* 학습의 척도로 Loss를 사용한다.
* Loss는 정답 데이터와 신경망 예측 결과를 비교한 단일 스칼라값으로 구할 수 있다.

신경망의 손실은 **손실 함수(loss function)**을 사용해서 구한다.

다중 클래스 분류(multi-class classification) 신경망에서는 손실함수로 흔히 **교차 엔트로피 오차(Cross Entropy Error)**를 이용한다.

**손실함수를 적용한 신경망 계층 구성**

![alt text](./img/TwoLayerNetwork-with-LossFunction.PNG "TwoLayerNet-with-LossFunction")

-X: 입력데이터<br>
-t: 정답레이블<br>
-L: 손실<br>
-Softmax계층의 출력: 확률<br>
-Cross Entropy Error 계층: Softmax 계층 출력 확률과 정답 레이블이 입력

**Softmax 함수**<br>

![alt text](./img/Softmax-function.PNG)

* Softmax함수 출력의 각 원소는 0.0이상 1.0이하의 실수이다. <br>
* 원소들을 모두 더하면 1.0이 된다.
* 이 때문에 Softmax의 출력을 '확률'로 해석할 수 있다.

In [20]:
import numpy as np

a = np.array([0.3, 2.9, 4.0])
exp_a = np.exp(a)
print(exp_a)

sum_exp_a = np.sum(exp_a)
print(sum_exp_a)

y = exp_a / sum_exp_a
print(y)

[ 1.34985881 18.17414537 54.59815003]
74.1221542101633
[0.01821127 0.24519181 0.73659691]


* Softmax 함수 구현

In [21]:
# Softmax 함수 정의

def softmax(s):
    c = np.max(s)
    exp_s = np.exp(s-c)
    sum_exp_s = np.sum(exp_s)
    y = exp_s / sum_exp_s
    return y

In [22]:
s = np.array([0.3, 2.9, 4.0])

print(softmax(s))

[0.01821127 0.24519181 0.73659691]


* Softmax 함수의 특징
    * 함수의 출력은 0에서 1.0사이이다.
    * 함수 출력의 총 합은 1이다.
    * Softmax함수를 적용해도 각 원소의 대소 관계는 변하지 않는다.

**CEE(Cross Entropy Error)**<br>
![alt text](./img/Cross-Entropy.PNG "Cross-Entropy function")
* tk는 k번째 클래스에 해당하는 정답 레이블이다.<br>
* log는 네이피어 상수(혹은 오일러의수)e를 밑으로 하는 로그이다.<br>
* 정답 레이블은 t=[0, 0, 1]과 같이 원-핫 벡터로 표현한다.

**Minibatch를 적용한 CrossEntropyError**<br>
![alt text](./img/Minibatch-CrossEntropy.PNG)

* tnk는 n번째 데이터에서 k차원째의 값을 의미한다.<br>
* ynk는 신경망의 출력이고, tnk는 정답 레이블이다.

**Softmax-with-Loss Layer**<br>
![alt text](./img/Softmax-with-Loss-Layer.png)

본 책에서는 소프트맥스 함수와 교차 엔트로피 오차를 계산하는 계층을 Softmax with Loss 계층 하나로 구현한다.

## 1.3.2 미분과 기울기

신경망 학습의 목표는 손실을 최소화하는 매개변수를 찾는 것이다.

이 때 중요한 것이 바로 **'미분'**과 **'기울기'**이다.

이번 절에서는 미분과 기울기에 대해 간략히 설명한다.

* Gradient(기울기)란 무엇인가
-x에 관한 y의 미분은 dy/dx라고 쓴다<br>
-이것이 의미하는 것은 x를 조금 변화시켰을 때(조금의 변화를 극한까지 줄일 때) y값이 얼마나 변하는가이다.

* 두 가지의 Gradient
    * 벡터의 각 원소에 대한 미분을 정리
    * 행렬에서의 기울기

* 벡터의 기울기

L은 스칼라, x는 벡터인 함수 L = f(x) 가 있다.

이 때 xi(x의 i번 째 원소)에 대한 미분과 x의 다른 원소의 미분을 다음과 같이 정리할 수 있다.

![alt text](./img/gradient-vector.PNG)

이처럼 벡터의 각 원소에 대한 미분을 정리한 것이 기울기이다.

* 행렬의 기울기

![alt text](./img/gradient-matrix.PNG)

* 행렬의 기울기는 원래의 행렬과 형상이 같다.
  
이 성질을 이용하면 매개변수 갱신과 연쇄 법칙을 쉽게 구현할 수 있다.

### 1.3.3 연쇄 법칙

학습 시 신경망은 학습데이터를 통해 손실(Loss)을 출력한다. 이 때 우리가 알고 싶은 것은 바로 **매개변수에 대한 손실의 기울기**이다.

Gradient of parameter?

* 연쇄 법칙(Chain-Rule)

오차역전파법을 이해하는 열쇠는 **연쇄법칙(Chain-Rule)**이다. 연쇄법칙이란 합성함수에 대한 미분의 법칙이다.

* 연쇄법칙이 중요한 이유

아무리 복잡하고, 많은 함수를 사용하더라도 개별 미분들을 이용해 전체 미분값을 구할 수 있기 때문이다.<br>
즉, 각 함수의 국소적 미분을 구할 수 있다면 그 값들을 곱해서 전체 미분을 구할 수 있다.

* Chain-Rule을 활용한 역전파 미분값 구하기

Chain-Rule에 따르면 역전파로 흐르는 미분값은 상류로부터 흘러온 미분과 각 연산 노드의 국소적인 미분을 곱해 계산할 수 있다.

## 1.3.4 계산 그래프

### 분기노드

![alt text](./img/분기노드.PNG)

분기노드는 위의 그림과 같이 분기하는 노드이다.

분기노드의 역전파는 (상류노드에서 흘러들어온) 분기노드의 합이다.

### Repeat Node

2개의 분기 노드를 일반화 하면 N개의 분기가 된다. 이를 Repeat Node라고 한다.

![alt text](./img/Repeat-Node.PNG)

분기노드 코드 구현

In [23]:
import numpy as np

D, N = 8, 7
x = np.random.randn(1, D)  # 입력
# y = np.random.randn(8, 7)
y = np.repeat(x, N, axis=0)  # 순전파

# 역전파
dy = np.random.randn(N,D)
dx = np.sum(dy, axis=0, keepdims=True)  # Keppdims= Ture
                                        # If this is set to True, the axes which are reduced are left
                                        # in the result as dimensions with size one. With this option,
                                        # the result will broadcast correctly against the input array.

### Sum Node

![alt text](./img/Sum-Node.PNG)

Sum Node는 범용 덧셈 노드이다. 

In [24]:
# sum node 구하기

import numpy as np

D, N = 8, 7
x = np.random.randn(D, N)
y = np.sum(x, axis=0, keepdims=True)  # keepdims=True를 통해 2차원 배열의 차원수를 유지한다.

dy = np.random.randn(1, D)
dx = np.repeat(dy, N, axis=0)

### MatMul Node

![alt text](./img/Minibatch-MatMul-Shape.JPG)

### MatMul Node 구현하기
이 노드를 하나의 계층으로 구현해보자

In [25]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
        
    def forward(self, x):
        W, = self.params
        out = np.matmul(x, W)
        self.x = x
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        
        self.grads[0][...] = dW  # 얕은 복사냐 깊은 복사냐? 둘 다 무슨 말이야??
        
        return dx

라인 18번 째 주석, 얕은 복사?? 깊은 복사??



In [26]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])


이 상태에서 a=b와 a[...]=b 모두 a에는 [4, 5, 6]이 할당된다.

그러나 두 경우에 a가 가리키는 메모리의 위치는 서로 다르다

![alt text](./img/얕은복사-깊은복사.PNG)

a = b에서는 a가 가리키는 메모리 위치가 b가 가리키는 위치와 같아진다. 

실제 데이터(4, 5, 6)는 복사되지 않는다는 뜻으로 이를 '얕은 복사'라고 한다.

한편 a[...]=b일 때는 a의 메모리 위치는 변하지 않고, 대신 a가 가리키는 메모리에 b의 원소가 복제된다. 

실제 데이터가 복제된다는 뜻에서 깊은 복사라고 한다.

'생략 기호'를 이용하여 변수의 메모리 주소를 고정할 수 있다.

이처럼 메모리 주소를 고정함으로써 인스턴스 변수 grads를 다루기 더 쉬워진다.

* grads list
    * grads list는 각 매개변수의 기울기를 저장한다.
    * list의 각 원소는 NumPy 배열이며, 계층을 생성할 때 한 번만 생성한다.
    * 항상 '생략기호'를 이용하므로 NumPy 배열의 메모리 주소가 변하는 일 없이 항상 값을 덮어쓴다.
    * 이렇게 하면 기울기를 그룹화 하는 작업을 최초에 한 번만 하면 된다는 이점이 생긴다.

## 1.3.5 기울기 도출과 역전파 구현

이번 절에서는 Sigmoid 계층, 완전연결 계층의 Affine 계층, Softmax with Loss 계층을 구현한다.

### Sigmoid 계층

In [28]:
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

### Affine 계층

In [29]:
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
        
    def forward(self, x):
        W, b = self.params
        out = np.matmul(x, W) + b
        self.x = x
        return out
    
    def backward(self, dout):
        W, b = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        db = np.sum(dout, axis=0)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

### Softmax with Loss 계층

소프트맥스 함수와 교차 엔트로피 오차는 Softmax with Loss라는 하나의 계층으로 구현한다.

![alt text](./img/Softmax-with-Loss-Layer.PNG)

위의 계산그래프에서 Softmax Function은 Softmax Layer로, Cross Entropy Error는 Cross Entropy Layer로 표기했다.

3-클래스 분류를 가정하여 이전 계층으로부터 3개의 입력을 받도록 했다.

Cross Entropy Error계층은 Softmax의 출력 (y1, y2, y3)와 정답 레이블 (t1, t2, t3)를 받고 이 데이터로 손실(Loss) L 출력한다.

**SoftmaxWithLoss계층 구현하기**

* Softmax 계층 구하기

In [31]:
def softmax(x):
    if x.ndim == 2:
        x = x - x.max(axis=1, keepdims=True)
        x = np.exp(x)
        x /= x.sum(axis=1, keepdims=True)
    elif x.ndim == 1:
        x = x - np.max(x)
        x = np.exp(x) / np.sum(np.exp(x))

    return x


class Softmax:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None
        
    def forward(self, x):
        '''
        c = np.max(x)
        exp_s = x-c
        sum_exp_s = np.sum(exp_s, axis=0)
        self.out = exp_s / sum_exp_s
        '''
        self.out = softmax(x)
        return self.out
    
    def backward(self, dout):
        dx = self.out * dout
        sumdx = np.sum(dx, axis=1, keepdims=True)
        dx -= self.out * sumdx
        return dx

In [33]:
class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y = None  # softmax의 출력
        self.t = None  # 정답 레이블
        
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        
        # 정답 레이블이 원핫 벡터일 정우 정답의 인덱스로 변환??
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)
            
        loss = cross_entropy_error(self.y, self.t)
        return loss
        
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        
        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1
        dx *= dout
        dx = dx / batch_size
        
        return dx

와... 이건 좀 어려운데...;;  -20.03.10.Tue. pm 6:10 -

==> [밑바닥부터 시작하는 딥러닝 1권] Appendix-A: Softmax-with-Loss 계층의 계산 그래프 공부합시다...!

## 1.3.6 가중치 갱신

오차역전파법으로 구한 기울기는 어디에 사용되는걸까?

* 신경망 학습 순서
    * 1단계: 미니 배치
    * 2단계: 기울기 계산
    * 3단계: 매개변수 갱신
    * 4단계: 반복

오차역전파법을 통해 각 매개변수에 대한 손실함수의 기울기를 계산한다 -> 기울기를 통해 어떤 매개변수가 손실에 영향을 많이 주는지 알 수 있다?!

* 오차역전파법을 통해 구한 매개변수에 대한 손실함수의 기울기
    * 이 기울기는 현재의 가중치 매개변수에서 손실을 가장 크게 하는 방향을 가리킨다.
    * 따라서 **매개변수를 그 기울기와 반대 방향으로 갱신**하면 손실을 줄일 수 있다.
    * 이것이 바로 **경사 하강법**(Gradient Descent)이다.

![alt text](./img/Gradient-Descent.PNG)

In [34]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]

## 정리